In [2]:
import pandas as pd
import numpy as np
import csv
from scipy.stats import norm
import streamlit as st



In [3]:
file_name = "2022-01-08.csv" 
dataset = pd.read_csv(file_name)
    



In [ ]:
dataset.head(10)

In [71]:
clean_data  = dataset.drop(["URL", "Thumbnail"], axis = 1) 
clean_data = clean_data.rename(columns = {"Unnamed: 0" : "Game Index", "Bayes average": "Bayes Average", "Users rated" : "Number_of_Ratings"})

# Cleaned the data: took out the Bayes average, URL and thumbnail columns

clean_data

,Game Index,ID,Name,Year,Rank,Average,Bayes Average,Number_of_Ratings
0,105,30549,Pandemic,2008,106,7.59,7.487,108975
1,189,822,Carcassonne,2000,190,7.42,7.309,108738
2,428,13,Catan,1995,429,7.14,6.970,108024
3,72,68448,7 Wonders,2010,73,7.74,7.634,89982
4,103,36218,Dominion,2008,104,7.61,7.499,81561
...,...,...,...,...,...,...,...,...
21826,12748,296892,Sacred Rites,2020,12749,7.37,5.531,30
21827,15016,217378,BABEL,2016,15017,6.61,5.517,30
21828,19459,18063,Goldrush,1970,19460,5.00,5.488,30
21829,17256,10052,Bongo Kongo,1989,17257,5.80,5.505,30


In [ ]:
# Data sorted by the average score
data_by_average = clean_data.sort_values("Average", ascending= False)

# I could move it through the following method, but I won't do it yet: 
# # Move column "C" to the first position
# column = data.pop("C")
# data.insert(0, "C", column)
# print(data)

data_by_average

In [ ]:
# Data sorted by the rank, same as sorting by the Bayes Average
data_by_rank = clean_data.sort_values("Rank")
data_by_rank

In [ ]:
data_sorted_by_bayes = clean_data.sort_values("Bayes Average", ascending = False)
data_sorted_by_bayes

In [7]:
file_reviews = "bgg-19m-reviews.csv" # second dataset
dataset_reviews = pd.read_csv(file_reviews)
# dataset_reviews.head(10)
# takes 45 secs to run, tough

In [ ]:
dataset_reviews.head(10)

In [8]:
# Clean the second dataset
# The unnamed section is kind of useless, and so are the user names
# the comments could be useful but 
clean_review_data = dataset_reviews.drop(["Unnamed: 0", "comment"], axis = 1)
clean_review_data = clean_review_data.rename(columns = {"user":"User", "rating":"Rating", "name" : "Name_of_Game"})

In [ ]:
clean_review_data
# count the number of times a game (an ID) has a positive review (> 6) 
# for each ID and divide by the total number of reviews.
# join after, subquery 

In [ ]:
clean_review_data.loc[(clean_review_data["ID"] == 1752) & (clean_review_data['Rating'] >= 6.0)]


In [9]:
# Finding the number of positive reviews per game:
def count_positive_reviews_per_game(dataset, ID):
    
    df_positive_reviews = dataset.loc[(dataset["ID"] == ID) & (dataset['Rating'] >= 6.0)] 
    # Loc Accesses a group of columns by labels and makes a new df
    positive_reviews_count = df_positive_reviews['Rating'].count() 
    # the variable counts every element in the df, so all the positive reviews
    
    
    return int(positive_reviews_count)
count_positive_reviews_per_game(clean_review_data, 30549 )

102640

In [10]:
# Function to get the total number of ratings for any game out of any dataset.
def total_reviews(dataset, ID):
    # Identify the column with the total number of ratings for every game (yields a series)
    total_reviews = dataset['Number_of_Ratings']
    # get specific row of the first ID, I might not even need this line...
    total_reviews_ID = total_reviews.loc[(dataset['ID']== ID)]
    return int(total_reviews_ID.iloc[0])

total_reviews(clean_data, 30549)

108975

In [11]:
def share_of_positive_reviews_per_game(ID):
    share_of_pos_reviews_per_game = float(count_positive_reviews_per_game(clean_review_data, ID) / total_reviews(clean_data, ID))
    return round(share_of_pos_reviews_per_game, 3)

share_of_positive_reviews_per_game(30549)


0.942

In [ ]:
# trial run for share of positive reviews for each game
for id in clean_data['ID']:
    print(share_of_positive_reviews_per_game(id))



In [12]:
def confidence_level(confidence):
    cl = norm.ppf(1- (1 - confidence)/2)
    return cl
# https://stackoverflow.com/questions/20864847/probability-to-z-score-and-vice-versa

def Wilson_Function(ID, confidence = .95):
    z = confidence_level(confidence)
    n = total_reviews(clean_data, ID)
    p_hat = 1 * share_of_positive_reviews_per_game(ID) #/ total_reviews(clean_data, ID)
    
    wilson_score = round(float((p_hat + z**2/(2*n) - z*np.sqrt((p_hat*(1 - p_hat) + z**2/(4*n))/n)) / (1 + (z**2)/n)), 2)
    


    return wilson_score
Wilson_Function(30549 , .95)

0.94

In [13]:
def array_scores(dataset, ID):
    list_scores = []
    for i in range(1,11):
        # return rows for my specific ID with the rating being between certain values.
        number_reviews_for_each_rating = dataset.loc[(dataset['ID'] == ID) & (dataset['Rating'].between(i, i + 1, inclusive='left'))]
        i_reviews = number_reviews_for_each_rating['Rating'].count()
        
        
        list_scores.append(int(i_reviews))
    
    
    return list_scores
array_scores(clean_review_data, 1752)


[2, 2, 1, 6, 9, 5, 4, 1, 0, 0]

In [14]:
def Bayesian_Average(ID, dataset = clean_review_data, confidence = .95):
    z = confidence_level(confidence)
    n = array_scores(dataset, ID)
    N = sum(n)
    first_part = 0
    second_part = 0
    for k, n_k in enumerate(n):
        first_part += (k+1)*(n[k]+1)/(N + 10)
        second_part += (k+1)*(k+1)*(n[k]+1)/(N+10)
    score = first_part - z*np.sqrt((second_part - first_part*first_part)/(N+11))
    
    return round(float(score), 2)

Bayesian_Average(1752)
# I need a better understanding of wtf this does. but also, I need to add it to the columns and graph it. I guess that's a tomorrow
# morning kind of job.
# then I want to implement all the rest of the stuff. the comments, which requires the new dataset and all...


4.33

In [ ]:
def New_Column(dataset, function):
    # create an empty list (our future column)
    New_column = []
    # for each row in a dataset I have to find its ID and apply a the funciton over it
    for id in dataset['ID'].unique():
        New_column.append(function(id))
    
    return New_column



In [120]:
# Adding a column to a dataset
def add_new_column_to_dataset(dataset_input, dataset_output, function, title_of_column = 'New Column'):
    # create an empty list (our future column)
    New_column = []
    # for each row in a dataset I have to find its ID and apply a the funciton over it
    for id in dataset_input['ID'].unique():
        New_column.append(function(id))
        
    dataset_output.insert(0, title_of_column, New_column)
    
    return dataset_output

# A function to create a database for each section of my data (most, mid, and leaset reviewed)
def clean_review_data_x_reviewed(output_database, global_database = clean_review_data):
    desired_ids = output_database['ID']
    clean_review_data_x_reviewed = global_database[global_database['ID'].isin(desired_ids)]
    return clean_review_data_x_reviewed

In [ ]:
most_reviewed_games = clean_data[clean_data['Number_of_Ratings'] > 25000] # Top 100 games, this way. (65000 for top 11)
add_new_column_to_dataset(clean_review_data_x_reviewed(most_reviewed_games),  most_reviewed_games, Bayesian_Average, 'New_Bayesian_Average') 
most_reviewed_games.sort_values('New_Bayesian_Average', ascending = False)

In [ ]:
# LEAST Reviewed Games
least_reviewed_games = clean_data[clean_data['Number_of_Ratings']== 50] # Bottom 164 games, possibly change this interval, overall pretty goo job though.

add_new_column_to_dataset(clean_review_data_x_reviewed(least_reviewed_games), least_reviewed_games, Bayesian_Average, 'New_Bayesian_Average')

least_reviewed_games.sort_values('New_Bayesian_Average', ascending = False)
# you could look at this top ten and break down the comments for the top 10, now that's actually kind of cool.
# can organize them by year

In [ ]:
# MID Reviewed Games
mid_reviewed_games = clean_data[clean_data['Number_of_Ratings'].between(3500, 4000)] #65000 for top 11.
#len(clean_data2)

add_new_column_to_dataset(clean_review_data_x_reviewed(mid_reviewed_games), mid_reviewed_games, Bayesian_Average, 'New_Bayesian_Average')
mid_reviewed_games.sort_values('New_Bayesian_Average', ascending = False)

,New_Bayesian_Average,Game Index,ID,Name,Year,Rank,Average,Bayes Average,Number_of_Ratings
891,6.62,557,180511,Shakespeare,2015,558,7.43,6.842,3998
892,7.27,539,1513,The Republic of Rome,1990,540,7.54,6.863,3997
893,7.41,1587,180020,Virus!,2015,1588,6.76,6.261,3990
894,6.61,1362,252929,Planet,2018,1363,6.75,6.349,3989
895,6.75,1105,132372,Guildhall,2012,1106,6.91,6.483,3984
...,...,...,...,...,...,...,...,...,...
1004,6.27,843,244331,Blue Lagoon,2018,844,7.19,6.636,3521
1005,7.71,378,215341,Thunderstone Quest,2018,379,7.86,7.038,3515
1006,7.23,2026,382,Heimlich & Co.,1984,2027,6.44,6.125,3509
1007,6.31,803,204472,Sub Terra,2017,804,7.38,6.672,3506


In [ ]:
# Motivation for range of mid_reviewed_games:

# 13.6k games between 30 reviews and 200 reviews
# 5.3k games between 200 reviews and a 1000 reviews
# 1.3 k games between 1000 and 2200 reviews
# the average being 866 and there being 130000 games with very few reviews I thought 3500-4000
average_ratings_per_game = clean_data['Number_of_Ratings']
average_ratings_dataset = int(np.average(average_ratings_per_game))
average_ratings_dataset

In [ ]:
# Plotting the New_Bayesian_Average vs the common average for the top rated games
# How tf you use matplotlib lmao.